# NHL Teams data from NHL Stats API

Dataset generated from a JSON received from the NHL Stats API, contains response to the request for all NHL franchises.

For details, see notebook `notebooks/feature_extraction/nhl_api.ipynb`.

# Data cleanup and feature extraction
Only basic cleanup was performed via this notebook, more detailed cleanup was done using Tableau Prep.
## Cleanup plan
1. Extract conference id
2. Extract division id
3. Extract venue name

In [25]:
import json
import psycopg2
from contextlib import closing
import pandas as pd
from time import time
import os

In [2]:
from psycopg2.extras import DictCursor
os.chdir('Documents/repos/nhl_draft/')
os.listdir()

['.git',
 '.gitattributes',
 '.gitignore',
 '.idea',
 'auxil',
 'Bookmarks',
 'data',
 'Datasources',
 'Extensions',
 'Flows',
 'img',
 'Logs',
 'machineId.txt',
 'main.py',
 'Mapsources',
 'methodology',
 'models',
 'MRUMaestroFlows.json',
 'notebooks',
 'out',
 'README.md',
 'references',
 'reports',
 'requirements.txt',
 'Services',
 'Shapes',
 'SQL',
 'src',
 'Workbooks']

In [3]:
teams_api_data_path = 'data/nhl_api/teams.csv'
t = time()
df = pd.read_csv(teams_api_data_path)
# df = df.rename(columns={'Unnamed: 0': 'id'})
elapsed = time() - t
print("----- DataFrame with NHL Draft Data loaded"
      "\nin {0:.2f} seconds".format(elapsed) + 
      "\nwith {0:,} rows\nand {1:,} columns"
      .format(df.shape[0], df.shape[1]) + 
      "\n-- Column names:\n", df.columns)

----- DataFrame with NHL Draft Data loaded
in 0.05 seconds
with 31 rows
and 15 columns
-- Column names:
 Index(['abbreviation', 'active', 'conference', 'division', 'firstYearOfPlay',
       'franchise', 'franchiseId', 'id', 'link', 'locationName', 'name',
       'officialSiteUrl', 'shortName', 'teamName', 'venue'],
      dtype='object')


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 15 columns):
abbreviation       31 non-null object
active             31 non-null bool
conference         31 non-null object
division           31 non-null object
firstYearOfPlay    31 non-null int64
franchise          31 non-null object
franchiseId        31 non-null int64
id                 31 non-null int64
link               31 non-null object
locationName       31 non-null object
name               31 non-null object
officialSiteUrl    31 non-null object
shortName          31 non-null object
teamName           31 non-null object
venue              31 non-null object
dtypes: bool(1), int64(3), object(11)
memory usage: 3.5+ KB


## Extract conference id

In [ ]:
col = 'conference'
new_var = 'id'
df[col + '_json'] = df[col].str.replace("'", "\"")
df[col + '_json'] = \
    df[col + '_json'].apply(lambda x: json.loads(x))
df[col + '_' + new_var] = \
    df[col + '_json'].apply(lambda x: x[new_var])
print("{0} {1} extracted!".format(col.title(), new_var))

## Extract division id

In [ ]:
col = 'division'
new_var = 'id'
df[col + '_json'] = df[col].str.replace("'", "\"")
df[col + '_json'] = \
    df[col + '_json'].apply(lambda x: json.loads(x))
df[col + '_' + new_var] = \
    df[col + '_json'].apply(lambda x: x[new_var])
print("{0} {1} extracted!".format(col.title(), new_var))

## Extract division id

In [18]:
col = 'venue'
new_var = 'name'
df[col + '_json'] = df[col].str.replace("'", "\"")
df[col + '_json'] = \
    df[col + '_json'].apply(lambda x: json.loads(x))
df[col + '_' + new_var] = \
    df[col + '_json'].apply(lambda x: x[new_var])
print("{0} {1} extracted!".format(col.title(), new_var))

Venue name extracted!


## Record results to PostgreSQL database

In [36]:
with closing(psycopg2.connect(
    dbname='nhl_draft', user='postgres',
    password='postgres', host='localhost'
)) as conn:
    with conn.cursor() as cur:
        cur.execute(
            """
drop table nhl_draft.nhl_draft.draft_players_all;
        
create table nhl_draft.nhl_draft.draft_players_all
(
    csplayerid integer,
    name       varchar(30),
    nat        char(3),
    b_date     date,
    b_place    varchar(50),
    b_city     varchar(30),
    b_state    char(2),
    b_country  varchar(30),
    ateam      varchar(50),
    league     varchar(30),
    year       date,
    dteam      integer,
    overall    integer,
    round      integer,
    in_round   integer,
    height     numeric,
    weight     numeric,
    pos        varchar(2),
    shoots     char,
    player_id  serial not null
        constraint draft_players_all_pk
            primary key
);

alter table nhl_draft.nhl_draft.draft_players_all
    owner to postgres;

create unique index draft_players_all_player_id_uindex
    on nhl_draft.nhl_draft.draft_players_all (player_id);
            """)
print("New table 'draft_players_all' created!")

New table 'draft_players_all' created!


## Record results to a new .csv file

In [19]:
save_path = 'data/nhl_api/teams_precleaned.csv'
t = time()
df.to_csv(save_path, index=False)
elapsed = time() - t
print("DataFrame saved to file:\n", save_path,
      "\ntook {0:.2f} seconds".format(elapsed))

DataFrame saved to file:
 data/nhl_api/teams_precleaned.csv 
took 0.16 seconds
